In [4]:
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
import pickle
from data_loader import get_loader 
from build_vocab import Vocabulary
from model import T_Att, DecoderRNN 
from torch.autograd import Variable 
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms
from PIL import Image

def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)


transform = transforms.Compose([ 
        transforms.Resize((224,224)), 
        transforms.ToTensor(), 
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])

with open("./data/vocab.pkl", 'rb') as f:
        vocab = pickle.load(f)
        
        

In [23]:
vocab.idx2word[3]

'<unk>'

In [2]:
data_loader = get_loader('../../train2014', "./annotations/captions_train2014.json", vocab, 
                             transform, 128,
                             shuffle=True, num_workers=2) 


loading annotations into memory...
Done (t=0.94s)
creating index...
index created!


In [3]:
for i, (images, captions, lengths) in enumerate(data_loader):
                print(captions)
                print('here')
                print(lengths)


    1     4  1168  ...   1281    19     2
    1     4   170  ...      2     0     0
    1     4    29  ...      0     0     0
       ...          ⋱          ...       
    1     4    99  ...      0     0     0
    1     4   170  ...      0     0     0
    1     4   170  ...      0     0     0
[torch.LongTensor of size 128x24]

here
[24, 22, 21, 20, 19, 18, 18, 18, 17, 17, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10]

    1     4    60  ...    499    19     2
    1   251    53  ...     19     2     0
    1    51  3846  ...      2     0     0
       ...          ⋱    


    1     4    34  ...   1074    19     2
    1   110    40  ...      0     0     0
    1   221    29  ...      0     0     0
       ...          ⋱          ...       
    1   139   207  ...      0     0     0
    1   170  1566  ...      0     0     0
    1     4   433  ...      0     0     0
[torch.LongTensor of size 128x25]

here
[25, 22, 22, 20, 20, 19, 19, 18, 18, 18, 18, 17, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9]

    1    13   112  ...    115    19     2
    1     4   254  ...      0     0     0
    1     4  3317  ...      0     0     0
       ...          ⋱     

Process Process-1:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/tushar/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/tushar/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/tushar/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/tushar/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/tushar/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/Users/tushar/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/Users/tushar/anaconda

KeyboardInterrupt: 

In [18]:
import sys
sys.path.append('cocoapi/PythonAPI')
from pycocotools.coco import COCO

coco = COCO('./annotations/captions_train2014.json')

loading annotations into memory...
Done (t=0.90s)
creating index...
index created!


In [19]:
ids = list(coco.anns.keys())
ids
coco.anns

{48: {'caption': 'A very clean and well decorated empty bathroom',
  'id': 48,
  'image_id': 318556},
 67: {'caption': 'A panoramic view of a kitchen and all of its appliances.',
  'id': 67,
  'image_id': 116100},
 126: {'caption': 'A blue and white bathroom with butterfly themed wall tiles.',
  'id': 126,
  'image_id': 318556},
 148: {'caption': 'A panoramic photo of a kitchen and dining room',
  'id': 148,
  'image_id': 116100},
 173: {'caption': 'A graffiti-ed stop sign across the street from a red car ',
  'id': 173,
  'image_id': 379340},
 188: {'caption': 'A vandalized stop sign and a red beetle on the road',
  'id': 188,
  'image_id': 379340},
 219: {'caption': 'A bathroom with a border of butterflies and blue paint on the walls above it.',
  'id': 219,
  'image_id': 318556},
 255: {'caption': 'An angled view of a beautifully decorated bathroom.',
  'id': 255,
  'image_id': 318556},
 272: {'caption': 'The two people are walking down the beach.',
  'id': 272,
  'image_id': 134754

In [10]:
caption = coco.anns[ids[48]]['caption']
caption

'A kitchen with a tile floor has cabinets with no doors, a dishwasher, a sink, and a refrigerator.'

In [11]:
coco.anns[ids[48]]

{'caption': 'A kitchen with a tile floor has cabinets with no doors, a dishwasher, a sink, and a refrigerator.',
 'id': 1558,
 'image_id': 299675}

In [20]:
coco = COCO('./annotations/instances_train2014.json')

loading annotations into memory...
Done (t=17.40s)
creating index...
index created!


In [13]:
ids = coco.anns.keys()

In [14]:
ids


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [17]:
coco.anns

{86: {'area': 54652.9556,
  'bbox': [116.95, 305.86, 285.3, 266.03],
  'category_id': 58,
  'id': 86,
  'image_id': 480023,
  'iscrowd': 0,
  'segmentation': [[312.29,
    562.89,
    402.25,
    511.49,
    400.96,
    425.38,
    398.39,
    372.69,
    388.11,
    332.85,
    318.71,
    325.14,
    295.58,
    305.86,
    269.88,
    314.86,
    258.31,
    337.99,
    217.19,
    321.29,
    182.49,
    343.13,
    141.37,
    348.27,
    132.37,
    358.55,
    159.36,
    377.83,
    116.95,
    421.53,
    167.07,
    499.92,
    232.61,
    560.32,
    300.72,
    571.89]]},
 89: {'area': 421.47274999999996,
  'bbox': [245.54, 208.17, 40.14, 19.1],
  'category_id': 58,
  'id': 89,
  'image_id': 50518,
  'iscrowd': 0,
  'segmentation': [[252.46,
    208.17,
    267.96,
    210.11,
    280.14,
    213.98,
    285.68,
    221.45,
    279.59,
    227.27,
    270.73,
    223.12,
    256.61,
    218.69,
    253.02,
    217.3,
    248.03,
    220.07,
    245.54,
    215.92,
    245.8

In [21]:
coco.info()

description: COCO 2014 Dataset
url: http://cocodataset.org
version: 1.0
year: 2014
contributor: COCO Consortium
date_created: 2017/09/01


In [9]:
import json

In [ ]:
class data_loader:
    def __init__(self):
        self.captions = json.load(open(".annotations/captions_train2014.json"))
        self.captions = json.load(open(".annotations/captions_train2014.json"))

In [5]:
import pandas as pd
df = pd.read_csv('dataset.csv', names = ['image_id','q_id','question','answer','caption'])

In [37]:
df.head()

,image_id,q_id,question,answer,caption
0,262145,262145000,Where is this taken?,market,People shop for vegetables like carrots and po...
1,262145,262145002,What veggies are shown?,carrots and potatoes,People shop for vegetables like carrots and po...
2,262146,262146000,What color is the snow?,white,A person on snow skis going down a steep slope.
3,262146,262146001,What is the person doing?,skiing,A person skiing down a steep snowy hill.
4,524291,524291000,What is in the person's hand?,frisbee,A dog watches a person who is holding a Frisbee.


In [42]:
df['image_id'][0]

262145

In [43]:
coco = COCO('/Users/tushar/Downloads/code/VQG/annotations/captions_train2014.json')

loading annotations into memory...
Done (t=0.88s)
creating index...
index created!


In [46]:
coco.loadImgs(262145)[0]['file_name']

'COCO_train2014_000000262145.jpg'

In [14]:
df2 = json.load(open('../data/v2_mscoco_train2014_annotations.json','r'))

In [17]:
df2['annotations'][0]

{'answer_type': 'other',
 'answers': [{'answer': 'net', 'answer_confidence': 'maybe', 'answer_id': 1},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 2},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 3},
  {'answer': 'netting', 'answer_confidence': 'yes', 'answer_id': 4},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 5},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 6},
  {'answer': 'mesh', 'answer_confidence': 'maybe', 'answer_id': 7},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 8},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 9},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 10}],
 'image_id': 458752,
 'multiple_choice_answer': 'net',
 'question_id': 458752000,
 'question_type': 'what is this'}

In [18]:
df2.keys()

dict_keys(['info', 'license', 'data_subtype', 'annotations', 'data_type'])

In [19]:
df2['data_type']

'mscoco'

In [5]:
import torch
import torchvision.transforms as transforms
import torch.utils.data as data
import os
import pickle
import numpy as np
import nltk
from PIL import Image
from build_vocab import Vocabulary
from pycocotools.coco import COCO
import pandas as pd

In [6]:
class VQG_Dataset(data.Dataset):
    
    """ Custom Dataset compatible with torch.utils.data.DataLoader."""
    
    def __init__(self, root, json, data, vocab, transform=None):
        """Set the path for images, captions and vocabulary wrapper.
        
        Args:
            root: image directory.
            json: coco annotation file path.
            data: preprocessed csv dataset file path
            vocab: vocabulary wrapper.
            transform: image transformer.
        """
        self.root = root
        self.coco = COCO(json)
        self.data = pd.read_csv(data, names = ['image_id','q_id','question','answer','caption'])
        self.ids = self.data.index.values
        self.vocab = vocab
        self.transform = transform
        
        
    def __getitem__(self, index):
        """Returns one data triplet (image, caption, QA)."""
        coco = self.coco
        vocab = self.vocab
        dataset = self.data
        im_id = dataset['image_id'][index]
        path = coco.loadImgs(int(im_id))[0]['file_name']
        caption = dataset['caption'][index]
        question = dataset['question'][index]
        answer = dataset['answer'][index]

        image = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        # Convert caption (string) to word ids.
        tokens = nltk.tokenize.word_tokenize(str(caption).lower())
        caption = []
        caption.append(vocab('<start>'))
        caption.extend([vocab(token) for token in tokens])
        caption.append(vocab('<end>'))
        target_caption = torch.Tensor(caption)

        # Combine question and answer (string) and convert to word ids.
        tokens = nltk.tokenize.word_tokenize(str(question).lower())
        QA = []
        QA.append(vocab('<start>'))
        QA.extend([vocab(token) for token in tokens])
        QA.append(vocab('<end>'))

        tokens = nltk.tokenize.word_tokenize(str(answer).lower())
        QA.append(vocab('<start>'))
        QA.extend([vocab(token) for token in tokens])
        QA.append(vocab('<end>'))

        target_QA = torch.Tensor(QA)
        return image, target_caption, target_QA
    
    def __len__(self):
        return len(self.ids)

In [7]:
transform = transforms.Compose([ 
        transforms.Resize((224,224)), 
        transforms.ToTensor(), 
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])
data_set = VQG_Dataset('/Users/tushar/Downloads/train2014','/Users/tushar/Downloads/code/VQG/annotations/captions_train2014.json','/Users/tushar/Downloads/code/VQG/dataset.csv', vocab, transform)

loading annotations into memory...
Done (t=0.88s)
creating index...
index created!


In [8]:
data_set[1]

(
 ( 0 ,.,.) = 
  -0.2513 -0.2513 -0.2171  ...  -0.1314 -0.1486 -0.1657
  -0.2513 -0.1999 -0.2342  ...  -0.1486 -0.1486 -0.1657
  -0.2513 -0.2684 -0.2342  ...  -0.1314 -0.1314 -0.1486
            ...             ⋱             ...          
   0.1939  0.5193  0.3481  ...   1.0159  2.0092  2.1633
   0.1254  0.3309  0.2453  ...   0.5536  1.8037  2.1462
   0.1426  0.3138  0.1597  ...   0.3652  1.3584  2.1119
 
 ( 1 ,.,.) = 
   0.5903  0.5903  0.5903  ...   0.6604  0.6429  0.6604
   0.5903  0.6078  0.6429  ...   0.6429  0.6429  0.6429
   0.6254  0.6429  0.6429  ...   0.6779  0.6604  0.6429
            ...             ⋱             ...          
  -0.8102 -0.4426 -0.6527  ...   1.0980  2.2360  2.3761
  -0.9678 -0.6877 -0.8277  ...   0.5903  2.0259  2.3761
  -1.0203 -0.7752 -0.8978  ...   0.4153  1.5532  2.3585
 
 ( 2 ,.,.) = 
   1.5420  1.5245  1.5420  ...   1.5071  1.5071  1.4722
   1.5420  1.5594  1.5594  ...   1.3677  1.3502  1.3502
   1.5071  1.5420  1.5942  ...   1.4722  1.4374  1.4374


In [4]:
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (image, caption).
    
    We should build custom collate_fn rather than using default collate_fn, 
    because merging caption (including padding) is not supported in default.
    Args:
        data: list of tuple (image, caption). 
            - image: torch tensor of shape (3, 256, 256).
            - caption: torch tensor of shape (?); variable length.
    Returns:
        images: torch tensor of shape (batch_size, 3, 256, 256).
        targets: torch tensor of shape (batch_size, padded_length).
        lengths: list; valid length for each padded caption.
    """
    # Sort a data list by caption length (descending order).
    data.sort(key=lambda x: len(x[2]), reverse=True)
    images, captions, qa = zip(*data)

    # Merge images (from tuple of 3D tensor to 4D tensor).
    images = torch.stack(images, 0)

    # Merge captions (from tuple of 1D tensor to 2D tensor).
    lengths_c = [len(cap) for cap in captions]
    targets_c = torch.zeros(len(captions), max(lengths_c)).long()
    for i, cap in enumerate(captions):
        end = lengths_c[i]
        targets_c[i, :end] = cap[:end] 
    
    # Merge qa (from tuple of 1D tensor to 2D tensor).
    lengths_q = [len(cap) for cap in qa]
    targets_q = torch.zeros(len(qa), max(lengths_q)).long()
    for i, cap in enumerate(qa):
        end = lengths_q[i]
        targets_q[i, :end] = cap[:end] 
    
    return images, targets_c, lengths_c, targets_q, lengths_q


In [10]:
from torch.utils.data import DataLoader
dataloader = DataLoader(data_set, batch_size=4,
                        shuffle=True, num_workers=2, collate_fn=collate_fn)


NameError: name 'collate_fn' is not defined

In [9]:
for i,(images,captions,cap_len,qa,qa_len) in enumerate(dataloader):
    print(images.size())
    print(captions.size())
    print(qa.size())
    break

NameError: name 'dataloader' is not defined

In [1]:
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
import pickle
import preprocess_get_model
import sys
sys.path.append('skip-thoughts.torch/pytorch')
from skipthoughts import UniSkip
import skimage.io as io
from data_loader import get_loader 
from build_vocab import Vocabulary
from model import T_Att, DecoderRNN 
from torch.autograd import Variable 
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms
from PIL import Image

def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)
    

# Image preprocessing
# For normalization, see https://github.com/pytorch/vision#models
transform = transforms.Compose([ 
    transforms.Resize((224,224)), 
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), 
                         (0.229, 0.224, 0.225))])

# Load vocabulary wrapper.
with open('./data/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

# Build data loader
data_loader = get_loader('/Users/tushar/Downloads/train2014', '/Users/tushar/Downloads/code/VQG/annotations/captions_train2014.json','/Users/tushar/Downloads/code/VQG/dataset.csv', vocab, transform, 32, True, 2) 

# Build the models
im_encoder = preprocess_get_model.model()
attention = T_Att()
decoder = DecoderRNN(512, 512, 
                     len(vocab), 1)

if torch.cuda.is_available():
    im_encoder.cuda()
    attention.cuda()
    decoder.cuda()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
params = list(decoder.parameters()) + list(attention.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)


loading annotations into memory...
Done (t=0.98s)
creating index...
index created!


In [6]:
for i, (images, captions, cap_lengths, qa, qa_lengths, vocab_words) in enumerate(data_loader):
            
    # Set mini-batch dataset
    images = to_var(images, volatile=True)
    captions = to_var(captions)
    qa = to_var(qa)
    targets = pack_padded_sequence(qa, qa_lengths, batch_first=True)[0]

    # Forward, Backward and Optimize
    decoder.zero_grad()
    attention.zero_grad()
    #features = encoder(images)
    img_embeddings = im_encoder(images)

    uni_vocab = []
    for word_id in vocab_words:
        word = vocab.idx2word[word_id]
        uni_vocab.append(word) 

    uniskip = UniSkip('/Users/tushar/Downloads/code/data/skip-thoughts', uni_vocab)
    print(captions.size())
    print(img_embeddings.size())
    print(cap_lengths)
    cap_embeddings = uniskip(captions, cap_lengths)
    ctx_vec = attention(img_embeddings,cap_embeddings)
    outputs = decoder(ctx_vec, qa, qa_lengths)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    break

torch.Size([32, 18])
torch.Size([32, 2048, 7, 7])
[13, 13, 15, 17, 14, 11, 14, 12, 14, 14, 14, 16, 16, 12, 12, 12, 14, 13, 10, 18, 11, 13, 15, 13, 10, 11, 14, 13, 14, 12, 13, 11]


RuntimeError: index out of range at /Users/soumith/minicondabuild3/conda-bld/pytorch_1512381214802/work/torch/lib/TH/generic/THTensorMath.c:277

In [5]:
np.unique(torch.tensor([1, 3, 2, 3], dtype=torch.Long))

AttributeError: module 'torch' has no attribute 'Long'

In [7]:
x = torch.rand((3,3)) * 10
np.unique(x.round().numpy())
torch.unique(x.round())

AttributeError: module 'torch' has no attribute 'unique'